In [1]:
import numpy as np
import pandas as pd

In [2]:
import json

In [3]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten
# from keras.optimizers import Adam

Using TensorFlow backend.


In [4]:
def load_embedding(filename='embedding.csv'):
    return pd.read_csv(filename, header=None)

In [5]:
emb = load_embedding()

In [6]:
emb.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.585270,0.250145,0.469886,0.247563,-0.016136,-0.228900,-0.45698,0.184238,-0.532028,0.214238,...,0.005357,0.182222,0.264911,0.145107,-0.020590,0.970097,-0.372763,-0.182743,0.757045,-0.681987
1,-0.584020,0.390310,0.652820,-0.340300,0.194930,-0.834890,0.11929,-0.572910,-0.568440,0.729890,...,0.285860,-0.052162,-0.508180,-0.634590,0.338890,0.284160,-0.203400,-1.233800,0.467150,0.788580
2,0.257690,0.456290,-0.769740,-0.376790,0.592720,-0.063527,0.20545,-0.573850,-0.290090,-0.136620,...,0.030498,-0.395430,-0.385150,-1.000200,0.087599,-0.310090,-0.346770,-0.314380,0.750040,0.970650
3,-0.039369,1.203600,0.354010,-0.559990,-0.520780,-0.669880,-0.75417,-0.653400,-0.232460,0.586860,...,-0.601410,0.504030,-0.083316,0.202390,0.443000,-0.060769,-0.428070,-0.084135,0.491640,0.085654
4,-0.268200,-0.478700,0.180990,-0.538370,-0.240210,-0.562030,0.20944,0.423580,-0.461470,0.769030,...,0.005399,-0.533440,-0.187060,0.522540,0.243610,0.051387,0.272100,-0.444330,0.019464,0.627820


In [7]:
emb_dim = emb.shape[1]

In [8]:
emb_dim

50

In [9]:
def load_dict(filename):
    with open(filename, 'r') as f:
        return json.load(f)

def load_index_word_map(word2ind_filename='word2ind', ind2word_filename='ind2word'):
    word2ind = load_dict(word2ind_filename)
    ind2word = load_dict(ind2word_filename)
    return word2ind, ind2word

In [10]:
word2ind, ind2word = load_index_word_map()

In [11]:
vocab_size = len(word2ind)

In [12]:
def load_training_samples(filename='train.csv'):
    return pd.read_csv(filename, header=None)

In [13]:
dataset = load_training_samples()

In [14]:
def train_valid_split(x, y, test_size=0.3, random_state=43):
    assert len(x) == len(y), 'Feature and label must have same length.'
    np.random.RandomState(seed=random_state)
    length = len(x)
    choices = list(range(length))
    val_choices = np.random.choice(choices, int(length*test_size), replace=False).tolist()
    train_choices = list(set(choices) - set(val_choices))
    return x[train_choices, :], x[val_choices, :], y[train_choices, :], y[val_choices, :]

In [15]:
train_x, val_x, train_y, val_y = train_valid_split(dataset.values[:, :-1], dataset.values[:, -1:], test_size=0.3, random_state=43)

In [16]:
train_x.shape

(1946113, 10)

In [19]:
train_y.shape

(1946113, 1)

In [20]:
val_y.shape

(834048, 1)

In [21]:
emb.shape

(32930, 50)

In [22]:
input_length = 10

In [23]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_length, weights=[emb]))
model.add(Dropout(.2))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(Dropout(.2))
model.add(LSTM(100, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(.2))
# model.add(Flatten())
model.add(Dense(vocab_size, activation='softmax'))

In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 50)            1646500   
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 100)           60400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 100)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
__________

In [25]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [27]:
model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=10, batch_size=10000)

Train on 1946113 samples, validate on 834048 samples
Epoch 1/10
1946113/1946113 [==============================] - 134s - loss: 6.1839 - acc: 0.1199 - val_loss: 5.5282 - val_acc: 0.1433
Epoch 2/10
1946113/1946113 [==============================] - 134s - loss: 5.3462 - acc: 0.1654 - val_loss: 4.9920 - val_acc: 0.1972
Epoch 3/10
1946113/1946113 [==============================] - 134s - loss: 5.0034 - acc: 0.1949 - val_loss: 4.7625 - val_acc: 0.2168
Epoch 4/10
1946113/1946113 [==============================] - 134s - loss: 4.8415 - acc: 0.2089 - val_loss: 4.6400 - val_acc: 0.2276
Epoch 5/10
1946113/1946113 [==============================] - 135s - loss: 4.7424 - acc: 0.2167 - val_loss: 4.5684 - val_acc: 0.2344
Epoch 6/10
1946113/1946113 [==============================] - 136s - loss: 4.6726 - acc: 0.2218 - val_loss: 4.5123 - val_acc: 0.2384
Epoch 7/10
1946113/1946113 [==============================] - 134s - loss: 4.6172 - acc: 0.2258 - val_loss: 4.4681 - val_acc: 0.2412
Epoch 8/10
19461

In [34]:
model.optimizer.lr = 0.01

In [35]:
model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=1, batch_size=200)

Train on 389061 samples, validate on 166740 samples
Epoch 1/1
389061/389061 [==============================] - 671s - loss: 4.4999 - acc: 0.2323 - val_loss: 4.3772 - val_acc: 0.2507


In [37]:
model.optimizer.lr = 0.001

In [38]:
model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=2, batch_size=200)

Train on 389061 samples, validate on 166740 samples
Epoch 1/2
389061/389061 [==============================] - 920s - loss: 4.0703 - acc: 0.2574 - val_loss: 4.2143 - val_acc: 0.2742
Epoch 2/2
389061/389061 [==============================] - 620s - loss: 3.6715 - acc: 0.2869 - val_loss: 4.1293 - val_acc: 0.2966


In [28]:
# save model
model.save('model20171228.h5')